# Attempt to fine tune Universal Sentence Encoder (transformer)

**Note:** Not working!!


In [0]:
!nvidia-smi

Sat Jun  6 14:53:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import tensorflow as tf
import tensorflow_hub as hub

In [0]:
import json
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd
import re


def clean_text(raw_text: str):
    if raw_text is None:
        return ''

    soup = BeautifulSoup(raw_text, features="html.parser")
    raw_text = soup.get_text()
    raw_text = raw_text.replace('\n', ' ').replace('\xa0', ' ')
    return raw_text


def read_json_as_df(path: str) -> pd.DataFrame:
    json_data = []

    with open(path, 'r', encoding='utf-8') as file:

        for line in file:
            data = json.loads(line)
            json_data.append([clean_text(data['post'].get('body', None)),
                              data['priority']])

    df = pd.DataFrame(data=json_data, columns=('text', 'priority'))

    return df

train_df = read_json_as_df('/content/drive/My Drive/temp_datasets/combined-train.json')
test_df = read_json_as_df('/content/drive/My Drive/temp_datasets/combined-test.json')

In [0]:
label_mapping = {
    'green': 0,
    'amber': 1,
    'escalate': 2,
    'red': 3
}

xtrain, ytrain = train_df.text.values, train_df.priority.map(label_mapping).values
xtest, ytest = test_df.text.values, test_df.priority.map(label_mapping).values

In [0]:
partial_x_train = xtrain[:5824]
x_val = xtrain[5824:]

partial_y_train = ytrain[:5824]
y_val = ytrain[5824:]

In [0]:
model = 'https://tfhub.dev/google/universal-sentence-encoder-large/5'

hub_layer = hub.KerasLayer(model, 
                           trainable=True, 
                           dtype=tf.string,
                           input_shape=[],
                           output_shape=[512])

In [0]:
model = tf.keras.Sequential()

model.add(hub_layer)

model.add(tf.keras.layers.Dense(128, activation='relu'))

model.add(tf.keras.layers.Dense(32, activation='relu'))

model.add(tf.keras.layers.Dense(4, name='output'))

model.build()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 512)               147354880 
_________________________________________________________________
dense (Dense)                (None, 128)               65664     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4128      
_________________________________________________________________
output (Dense)               (None, 4)                 132       
Total params: 147,424,804
Trainable params: 147,424,804
Non-trainable params: 0
_________________________________________________________________


In [0]:
opt = tf.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=opt,
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [0]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=30,
                    batch_size=8,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/30


728/728 [==============================] - 106s 146ms/step - loss: 1.0054 - accuracy: 0.6233 - val_loss: 1.1268 - val_accuracy: 0.5378
Epoch 2/30
728/728 [==============================] - 101s 139ms/step - loss: 1.0051 - accuracy: 0.6233 - val_loss: 1.1532 - val_accuracy: 0.5378
Epoch 3/30
728/728 [==============================] - 101s 138ms/step - loss: 1.0056 - accuracy: 0.6233 - val_loss: 1.1904 - val_accuracy: 0.5378
Epoch 4/30
728/728 [==============================] - 100s 138ms/step - loss: 1.0083 - accuracy: 0.6233 - val_loss: 1.0864 - val_accuracy: 0.5378
Epoch 5/30
728/728 [==============================] - 101s 138ms/step - loss: 1.0107 - accuracy: 0.6233 - val_loss: 1.0802 - val_accuracy: 0.5378
Epoch 6/30
728/728 [==============================] - 99s 137ms/step - loss: 1.0075 - accuracy: 0.6233 - val_loss: 1.1167 - val_accuracy: 0.5378
Epoch 7/30
728/728 [==============================] - 100s 137ms/step - loss: 1.0055 - accuracy: 0.6233 - val_loss: 1.0913 - val_accurac

In [0]:
results = model.evaluate(xtest, ytest)

print(results)

In [0]:
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [0]:
plt.clf()   # clear figure

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [0]:
# loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# def loss(model, x, y, training):
#   # training=training is needed only if there are layers with different
#   # behavior during training versus inference (e.g. Dropout).
#   y_ = model(x, training=training)

#   return loss_object(y_true=y, y_pred=y_)

# def grad(model, inputs, targets):
#   with tf.GradientTape() as tape:
#     loss_value = loss(model, inputs, targets, training=True)
#   return loss_value, tape.gradient(loss_value, model.trainable_variables)

# optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# train_loss_results = []
# train_accuracy_results = []

# num_epochs = 30

# for epoch in range(num_epochs):
#   epoch_loss_avg = tf.keras.metrics.Mean()
#   epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

#   # Training loop - using batches of 32
#   for x, y in zip(xtrain, ytrain):
#     # Optimize the model
#     loss_value, grads = grad(model, x, y)
#     optimizer.apply_gradients(zip(grads, model.trainable_variables))

#     # Track progress
#     epoch_loss_avg.update_state(loss_value)  # Add current batch loss
#     # Compare predicted label to actual label
#     # training=True is needed only if there are layers with different
#     # behavior during training versus inference (e.g. Dropout).
#     epoch_accuracy.update_state(y, model(x, training=True))

#   # End epoch
#   train_loss_results.append(epoch_loss_avg.result())
#   train_accuracy_results.append(epoch_accuracy.result())

#   if epoch % 50 == 0:
#     print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
#                                                                 epoch_loss_avg.result(),
#                                                                 epoch_accuracy.result()))